Hackathon-"is driver at fault?"

Analysing the fault of the driver based on the features

**AI20BTECH11010** *-haritha r*

In [1]:
import pandas as pd
import numpy as np
import io
from google.colab import files

#importing necessary libraries

In [2]:
####
#getting data set that would be used for training
# Upload CSV files from local drive and read them into a data frame
uploaded = files.upload()
data_train_set = pd.read_csv(io.BytesIO(uploaded['train.csv']))


Saving train.csv to train.csv


In [3]:
data_train=data_train_set #for convinence

In [4]:
data_train.shape

(51490, 42)

In [5]:
data_train.head() #printing data set structure

,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Off-Road Description,...,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Location,Fault
0,MP060D,10196,Montgomery County Police,Injury Crash,06/05/2017 04:27:00 PM,Maryland (State),OLD HUNDRED RD,County,THURSTON RD,12305 SHOREFIELD RD,...,No,No,2005,GMC,TRUCK,NO MISUSE,39.263378,-77.344203,3.6383 7.4233,1
1,MP000X,20080,Montgomery County Police,Injury Crash,02/22/2020 10:00:00 AM,County,JANET RD,County,FLACK ST,NaN,...,No,Yes,2018,FORD,FIESTA,NO MISUSE,39.068913,-77.063227,3.0862 7.324,1
2,MP070N,10103,Montgomery County Police,Property Damage Crash,06/11/2017 08:21:00 AM,Maryland (State),COLESVILLE RD,Maryland (State),STRUC #15082,NaN,...,No,No,2016,NEW FLYER,TBU,NO MISUSE,38.993578,-77.032718,3.9583 7.3783,0
3,MP130Y,10208,Montgomery County Police,Injury Crash,05/10/2019 07:30:00 AM,County,PARKLAND DR,County,FRANKFORT DR,NaN,...,No,No,2001,CHEV,SUBURBAN,OTHER,39.083480,-77.086933,"3.84,-70633",1
4,MP770Y,1091,Montgomery County Police,Injury Crash,08/02/2016 05:00:00 PM,Maryland (State),OLD GEORGETOWN RD,County,MCKINLEY ST,NaN,...,No,No,2013,KIA,RIO,NO MISUSE,38.996942,-77.109168,3.9917 7.0183,0


In [6]:
#There are observations that contain missing values
#Starting with defining a function to create a data frame of metadata containing count of null values and type.

def meta (dataframe):
    metadata = []
    for f in data_train.columns:
    
        # Counting null values
        null = data_train[f].isnull().sum()
    
        # Defining the data type 
        dtype = data_train[f].dtype
    
        # Creating a Dict that contains all the metadata for the variable
        f_dict = {
            'varname': f,
            'nulls':null,
            'dtype': dtype
        }
        metadata.append(f_dict)

    meta = pd.DataFrame(metadata, columns=['varname','nulls', 'dtype'])
    meta.set_index('varname', inplace=True)
    meta=meta.sort_values(by=['nulls'],ascending=False)
    return meta

In [7]:
meta(data_train)    #printing column names with number of nulls


,nulls,dtype
varname,,
Non-Motorist Substance Abuse,48106,object
Related Non-Motorist,47842,object
Off-Road Description,44540,object
Municipality,43923,object
Circumstance,39738,object
Equipment Problems,10666,object
Driver Substance Abuse,9095,object
Traffic Control,7774,object
Local Case Number,6453,object


In [8]:
missing_frac = data_train.isnull().mean() 
drop_list = sorted(missing_frac[missing_frac > 0.1].index)  #columns that contain missing values
print("columns that contain more missing values:",drop_list)
print("number of columns that contain more missing values:",len(drop_list))
 


columns that contain more missing values: ['Circumstance', 'Driver Substance Abuse', 'Equipment Problems', 'Local Case Number', 'Municipality', 'Non-Motorist Substance Abuse', 'Off-Road Description', 'Related Non-Motorist', 'Surface Condition', 'Traffic Control']
number of columns that contain more missing values: 10


In [9]:
def drop_cols(cols):
    data_train.drop(labels=cols, axis=1, inplace=True)      #dropping the columns with many missing value

drop_cols(drop_list)
data_train.shape

(51490, 32)

In [10]:
meta(data_train)   #note:now there are less missing values in any columns

,nulls,dtype
varname,,
Cross-Street Type,5019,object
Route Type,4968,object
Cross-Street Name,4522,object
Road Name,4502,object
Weather,4029,object
Drivers License State,2528,object
Vehicle Continuing Dir,1129,object
Vehicle Going Dir,1113,object
Vehicle Body Type,785,object


In [11]:
data_train['Fault'].value_counts()  #number of 1,0 s in fault


1    28330
0    23160
Name: Fault, dtype: int64

In [12]:
y_train = data_train.filter(['Fault']) #extracting the label set for training that is 'Fault' column
print(y_train.shape)


(51490, 1)


In [13]:
####
# Upload CSV files from local drive and read them into a data frame
uploaded = files.upload()
data_test_set = pd.read_csv(io.BytesIO(uploaded['test.csv']))
#getting the dataset that would be used for testing

Saving test.csv to test.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
data_test=data_test_set #for convinence

In [15]:
data_test.head()  #structure of the dataset


,Id,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,...,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Equipment Problems,Latitude,Longitude,Location
0,0,MP810M,NaN,MONTGOMERY,Property Damage Crash,12/16/2015 05:42:00 PM,County,WHITTIER BLVD,Maryland (State),WILSON LA,...,25,No,No,2004,LEXUS,RX330,NO MISUSE,38.988440,-77.127668,"3.84,-71763"
1,1,MP2001,NaN,Montgomery County Police,Property Damage Crash,05/01/2016 07:25:00 PM,County,RAILROAD ST,Municipality,E DIAMOND AVE,...,25,No,No,2001,MITSUBISHI,MIRAGE,NO MISUSE,39.094075,-77.205783,3.905 7.0733
2,2,MP6404,10125,Montgomery County Police,Property Damage Crash,03/16/2018 02:30:00 PM,Maryland (State),GEORGIA AVE,County,ARCOLA AVE,...,35,No,No,2016,FRIEGHTLINER,AMBULANCE,NO MISUSE,39.047180,-77.052688,"3.41,-70283"
3,3,MP0405,10270,Montgomery County Police,Property Damage Crash,05/25/2018 05:00:00 PM,NaN,NaN,NaN,NaN,...,25,No,No,2004,FORD,CN,NaN,39.178007,-77.238475,3.7067 7.345
4,4,MP090D,10374,Montgomery County Police,Property Damage Crash,09/23/2017 05:40:00 PM,Interstate (State),EISENHOWER MEMORIAL HWY,Maryland (State),MONTROSE RD,...,55,No,No,2008,UTIL,TL,UNKNOWN,39.056134,-77.154043,3.5143 7.5022


In [16]:
data_test.shape #shape of the dataset


(77235, 42)

In [17]:
def drop_cols_test(cols):
    data_test.drop(labels=cols, axis=1, inplace=True)        #dropping the columns that were dropped for training set in test set also

drop_cols_test(drop_list)  #the columns that contain missing values are same in training and test set
data_test.shape

(77235, 32)

In [18]:
meta(data_test) #note:now there are less missing values in any columns

,nulls,dtype
varname,,
Cross-Street Type,5019,object
Route Type,4968,object
Cross-Street Name,4522,object
Road Name,4502,object
Weather,4029,object
Drivers License State,2528,object
Vehicle Continuing Dir,1129,object
Vehicle Going Dir,1113,object
Vehicle Body Type,785,object


In [19]:
data_train=data_train.drop(['Fault'],axis=1)  #dropping the 'Fault' column to ensure pure training

In [20]:
data_train.head() #the training set

,Report Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Collision Type,Weather,...,Vehicle Going Dir,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Latitude,Longitude,Location
0,MP060D,Montgomery County Police,Injury Crash,06/05/2017 04:27:00 PM,Maryland (State),OLD HUNDRED RD,County,THURSTON RD,SINGLE VEHICLE,CLEAR,...,Unknown,40,No,No,2005,GMC,TRUCK,39.263378,-77.344203,3.6383 7.4233
1,MP000X,Montgomery County Police,Injury Crash,02/22/2020 10:00:00 AM,County,JANET RD,County,FLACK ST,STRAIGHT MOVEMENT ANGLE,CLEAR,...,South,25,No,Yes,2018,FORD,FIESTA,39.068913,-77.063227,3.0862 7.324
2,MP070N,Montgomery County Police,Property Damage Crash,06/11/2017 08:21:00 AM,Maryland (State),COLESVILLE RD,Maryland (State),STRUC #15082,SAME DIRECTION RIGHT TURN,CLEAR,...,West,35,No,No,2016,NEW FLYER,TBU,38.993578,-77.032718,3.9583 7.3783
3,MP130Y,Montgomery County Police,Injury Crash,05/10/2019 07:30:00 AM,County,PARKLAND DR,County,FRANKFORT DR,OTHER,CLOUDY,...,South,25,No,No,2001,CHEV,SUBURBAN,39.083480,-77.086933,"3.84,-70633"
4,MP770Y,Montgomery County Police,Injury Crash,08/02/2016 05:00:00 PM,Maryland (State),OLD GEORGETOWN RD,County,MCKINLEY ST,SAME DIR REAR END,CLEAR,...,South,40,No,No,2013,KIA,RIO,38.996942,-77.109168,3.9917 7.0183


In [21]:
data_test=data_test.drop(['Id'],axis=1)  #dropping the Id column in test dataset to match the structure of training set
data_test.head()  #the testing set

,Report Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Collision Type,Weather,...,Vehicle Going Dir,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Latitude,Longitude,Location
0,MP810M,MONTGOMERY,Property Damage Crash,12/16/2015 05:42:00 PM,County,WHITTIER BLVD,Maryland (State),WILSON LA,OTHER,CLEAR,...,North,25,No,No,2004,LEXUS,RX330,38.988440,-77.127668,"3.84,-71763"
1,MP2001,Montgomery County Police,Property Damage Crash,05/01/2016 07:25:00 PM,County,RAILROAD ST,Municipality,E DIAMOND AVE,SAME DIR REAR END,RAINING,...,West,25,No,No,2001,MITSUBISHI,MIRAGE,39.094075,-77.205783,3.905 7.0733
2,MP6404,Montgomery County Police,Property Damage Crash,03/16/2018 02:30:00 PM,Maryland (State),GEORGIA AVE,County,ARCOLA AVE,SAME DIRECTION SIDESWIPE,NaN,...,North,35,No,No,2016,FRIEGHTLINER,AMBULANCE,39.047180,-77.052688,"3.41,-70283"
3,MP0405,Montgomery County Police,Property Damage Crash,05/25/2018 05:00:00 PM,NaN,NaN,NaN,NaN,SAME DIRECTION SIDESWIPE,CLEAR,...,West,25,No,No,2004,FORD,CN,39.178007,-77.238475,3.7067 7.345
4,MP090D,Montgomery County Police,Property Damage Crash,09/23/2017 05:40:00 PM,Interstate (State),EISENHOWER MEMORIAL HWY,Maryland (State),MONTROSE RD,OTHER,CLEAR,...,North,55,No,No,2008,UTIL,TL,39.056134,-77.154043,3.5143 7.5022


In [22]:
test=data_test
train=data_train

In [23]:
test.head()

,Report Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Collision Type,Weather,...,Vehicle Going Dir,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Latitude,Longitude,Location
0,MP810M,MONTGOMERY,Property Damage Crash,12/16/2015 05:42:00 PM,County,WHITTIER BLVD,Maryland (State),WILSON LA,OTHER,CLEAR,...,North,25,No,No,2004,LEXUS,RX330,38.988440,-77.127668,"3.84,-71763"
1,MP2001,Montgomery County Police,Property Damage Crash,05/01/2016 07:25:00 PM,County,RAILROAD ST,Municipality,E DIAMOND AVE,SAME DIR REAR END,RAINING,...,West,25,No,No,2001,MITSUBISHI,MIRAGE,39.094075,-77.205783,3.905 7.0733
2,MP6404,Montgomery County Police,Property Damage Crash,03/16/2018 02:30:00 PM,Maryland (State),GEORGIA AVE,County,ARCOLA AVE,SAME DIRECTION SIDESWIPE,NaN,...,North,35,No,No,2016,FRIEGHTLINER,AMBULANCE,39.047180,-77.052688,"3.41,-70283"
3,MP0405,Montgomery County Police,Property Damage Crash,05/25/2018 05:00:00 PM,NaN,NaN,NaN,NaN,SAME DIRECTION SIDESWIPE,CLEAR,...,West,25,No,No,2004,FORD,CN,39.178007,-77.238475,3.7067 7.345
4,MP090D,Montgomery County Police,Property Damage Crash,09/23/2017 05:40:00 PM,Interstate (State),EISENHOWER MEMORIAL HWY,Maryland (State),MONTROSE RD,OTHER,CLEAR,...,North,55,No,No,2008,UTIL,TL,39.056134,-77.154043,3.5143 7.5022


In [24]:
train.head()

,Report Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Collision Type,Weather,...,Vehicle Going Dir,Speed Limit,Driverless Vehicle,Parked Vehicle,Vehicle Year,Vehicle Make,Vehicle Model,Latitude,Longitude,Location
0,MP060D,Montgomery County Police,Injury Crash,06/05/2017 04:27:00 PM,Maryland (State),OLD HUNDRED RD,County,THURSTON RD,SINGLE VEHICLE,CLEAR,...,Unknown,40,No,No,2005,GMC,TRUCK,39.263378,-77.344203,3.6383 7.4233
1,MP000X,Montgomery County Police,Injury Crash,02/22/2020 10:00:00 AM,County,JANET RD,County,FLACK ST,STRAIGHT MOVEMENT ANGLE,CLEAR,...,South,25,No,Yes,2018,FORD,FIESTA,39.068913,-77.063227,3.0862 7.324
2,MP070N,Montgomery County Police,Property Damage Crash,06/11/2017 08:21:00 AM,Maryland (State),COLESVILLE RD,Maryland (State),STRUC #15082,SAME DIRECTION RIGHT TURN,CLEAR,...,West,35,No,No,2016,NEW FLYER,TBU,38.993578,-77.032718,3.9583 7.3783
3,MP130Y,Montgomery County Police,Injury Crash,05/10/2019 07:30:00 AM,County,PARKLAND DR,County,FRANKFORT DR,OTHER,CLOUDY,...,South,25,No,No,2001,CHEV,SUBURBAN,39.083480,-77.086933,"3.84,-70633"
4,MP770Y,Montgomery County Police,Injury Crash,08/02/2016 05:00:00 PM,Maryland (State),OLD GEORGETOWN RD,County,MCKINLEY ST,SAME DIR REAR END,CLEAR,...,South,40,No,No,2013,KIA,RIO,38.996942,-77.109168,3.9917 7.0183


Feature processing

In [25]:
def add_datetime_info(dataset):
    #Convert to datetime format
    dataset['pickup_datetime'] = pd.to_datetime(dataset['Crash Date/Time'])
    dataset['hour'] = dataset.pickup_datetime.dt.hour
    dataset['day'] = dataset.pickup_datetime.dt.day
    dataset['month'] = dataset.pickup_datetime.dt.month
    dataset['weekday'] = dataset.pickup_datetime.dt.weekday
    dataset['year'] = dataset.pickup_datetime.dt.year
    dataset['minute'] = dataset.pickup_datetime.dt.minute
    dataset['second'] = dataset.pickup_datetime.dt.second
    return dataset

In [26]:
from sklearn import preprocessing
enc= preprocessing.LabelEncoder()
'''
cat_object_cols = ['Location','Report Number', 'Agency Name', 'Route Type', 'Road Name', 'Cross-Street Type','Cross-Street Name', 'Collision Type', 'Weather',
              'Light', 'Person ID','Drivers License State', 'Vehicle ID','Vehicle Damage Extent',
            'Vehicle First Impact Location','Vehicle Second Impact Location', 'Vehicle Body Type','Vehicle Movement','Vehicle Make', 'Vehicle Model',
            'Vehicle Continuing Dir', 'Vehicle Going Dir','Driverless Vehicle', 'Parked Vehicle','Injury Severity','ACRS Report Type']
'''
cat_object_cols = ['Report Number', 'Route Type', 'Road Name', 'Cross-Street Type','Cross-Street Name', 'Collision Type', 'Weather','Vehicle Make', 'Vehicle Model',
                    'Person ID','Drivers License State', 'Vehicle ID', 'Vehicle Body Type',]            
def preprocessing_objectmany(df_set): 
   for col_obj in cat_object_cols:
       df_set[col_obj] = df_set[col_obj].astype('str')
       df_set[col_obj] = enc.fit_transform(df_set[col_obj])
   return df_set

In [27]:
import re

def location_processing(data_frm):
  y=data_frm['Location']
  for i in range(len(y)):
        data_frm['Loc1'][i]=re.findall(r"[-+]?\d*\.\d+|\d+",str(y[i]))[0]
        data_frm['Loc2'][i]=re.findall(r"[-+]?\d*\.\d+|\d+",str(y[i]))[1]
  return data_frm


In [28]:
def pre_process(data):
   data['Vehicle Continuing Dir']=data['Vehicle Continuing Dir'].replace(to_replace=['North', 'South','East','West','Unknown'], value=[1, 2,3,4,5])
   data['Vehicle Going Dir']=data['Vehicle Going Dir'].replace(to_replace=['North', 'South','East','West','Unknown'], value=[1, 2,3,4,5])
   data['Light']=data['Light'].replace(to_replace=['DAYLIGHT','DARK LIGHTS ON','DARK NO LIGHTS',
                                                'DUSK','DAWN','DARK -- UNKNOWN LIGHTING','UNKNOWN','OTHER'],value=[1,2,3,4,5,6,7,8]) 
   data['Vehicle Damage Extent']=data['Vehicle Damage Extent'].replace(to_replace=['DISABLING','FUNCTIONAL','SUPERFICIAL',
                                                                                  'DESTROYED','NO DAMAGE','UNKNOWN','OTHER'],value=[1,2,3,4,5,6,7])    
   data['Vehicle First Impact Location']=data['Vehicle First Impact Location'].replace(to_replace=['TWELVE OCLOCK','SIX OCLOCK','ONE OCLOCK','ELEVEN OCLOCK',
                                                                                                   'TWO OCLOCK','TEN OCLOCK','SEVEN OCLOCK','FIVE OCLOCK','FOUR OCLOCK',
                                                                                                   'EIGHT OCLOCK','NINE OCLOCK','THREE OCLOCK','UNKNOWN','UNDERSIDE','NON-COLLISION',
                                                                                                   'ROOF TOP'],value=[12,6,1,11,2,10,7,5,4,8,9,3,13,14,15,16])
   data['Vehicle Second Impact Location']=data['Vehicle Second Impact Location'].replace(to_replace=['TWELVE OCLOCK','SIX OCLOCK','ONE OCLOCK','ELEVEN OCLOCK',
                                                                                                     'TWO OCLOCK','TEN OCLOCK','SEVEN OCLOCK','FIVE OCLOCK','FOUR OCLOCK',
                                                                                                     'EIGHT OCLOCK','NINE OCLOCK','THREE OCLOCK','UNKNOWN','UNDERSIDE',
                                                                                                     'NON-COLLISION','ROOF TOP'],value=[12,6,1,11,2,10,7,5,4,8,9,3,13,14,15,16])
   data['Vehicle Movement']=data['Vehicle Movement'].replace(to_replace=['MOVING CONSTANT SPEED','SLOWING OR STOPPING','STOPPED IN TRAFFIC LANE','MAKING LEFT TURN','ACCELERATING',
                                                                         'BACKING','MAKING RIGHT TURN','STARTING FROM LANE','CHANGING LANES','PARKED','UNKNOWN','PARKING',
                                                                         'STARTING FROM PARKED','MAKING U TURN','PASSING','OTHER','ENTERING TRAFFIC LANE','SKIDDING',
                                                                         'NEGOTIATING A CURVE','RIGHT TURN ON RED','LEAVING TRAFFIC LANE',
                                                                         'DRIVERLESS MOVING VEH.'],value=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22])
   data['Parked Vehicle']=data['Parked Vehicle'].replace(to_replace=['No','Yes'],value=[-1,1])
   data['Driverless Vehicle']=data['Driverless Vehicle'].replace(to_replace=['No','Unknown'],value=[-1,1])
   data['Injury Severity']=data['Injury Severity'].replace(to_replace=['NO APPARENT INJURY','POSSIBLE INJURY','SUSPECTED MINOR INJURY',
                                                                    'SUSPECTED SERIOUS INJURY','FATAL INJURY'],value=[1,2,3,4,5])
   data['ACRS Report Type']=data['ACRS Report Type'].replace(to_replace=['Property Damage Crash','Injury Crash','Fatal Crash'],value=[1,2,3])
   data['Agency Name']=data['Agency Name'].replace(to_replace=['Montgomery County Police','MONTGOMERY','Rockville Police Departme','Gaithersburg Police Depar',
                                                               'Takoma Park Police Depart','Maryland-National Capital','ROCKVILLE','GAITHERSBURG','MCPARK',
                                                               'TAKOMA'],value=[1,2,3,4,5,6,7,8,9,10])
   add_datetime_info(data)
   preprocessing_objectmany(data)
   location_processing(data)
   data['Vehicle Continuing Dir']=data['Vehicle Continuing Dir'].fillna(data['Vehicle Continuing Dir'].mean())
   data['Vehicle Going Dir']=data['Vehicle Going Dir'].fillna(data['Vehicle Going Dir'].mean())
   data['Light']=data['Light'].fillna(data['Light'].mean())
   data['Vehicle Damage Extent']=data['Vehicle Damage Extent'].fillna(data['Vehicle Damage Extent'].mean())
   data['Vehicle First Impact Location']=data['Vehicle First Impact Location'].fillna(data['Vehicle First Impact Location'].mean())
   data['Vehicle Second Impact Location']=data['Vehicle Second Impact Location'].fillna(data['Vehicle Second Impact Location'].mean())
   data['Vehicle Movement']=data['Vehicle Movement'].fillna(data['Vehicle Movement'].mean())
   return data


In [29]:
train["Loc1"] = np.nan
train["Loc2"] = np.nan
test["Loc1"] = np.nan
test["Loc2"] = np.nan

In [30]:
test=pre_process(test)
train=pre_process(train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [31]:
train=train.drop(['Crash Date/Time','pickup_datetime','Location'],axis=1)  
test=test.drop(['Crash Date/Time','pickup_datetime','Location'],axis=1)  


In [32]:
train.head() #final training set

,Report Number,Agency Name,ACRS Report Type,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Collision Type,Weather,Light,...,Longitude,Loc1,Loc2,hour,day,month,weekday,year,minute,second
0,359,1,2,3,1427,0,4109,15,2,1.0,...,-77.344203,3.6383,7.4233,16,5,6,0,2017,27,0
1,189,1,2,0,1008,0,1486,16,2,1.0,...,-77.063227,3.0862,7.3240,10,22,2,5,2020,0,0
2,398,1,1,3,424,3,3966,13,2,7.0,...,-77.032718,3.9583,7.3783,8,11,6,6,2017,21,0
3,593,1,2,0,1478,0,1540,7,3,1.0,...,-77.086933,3.8400,70633.0000,7,10,5,4,2019,30,0
4,2572,1,2,3,1425,0,2456,9,2,1.0,...,-77.109168,3.9917,7.0183,17,2,8,1,2016,0,0


In [33]:
test.head() #final test set

,Report Number,Agency Name,ACRS Report Type,Route Type,Road Name,Cross-Street Type,Cross-Street Name,Collision Type,Weather,Light,...,Longitude,Loc1,Loc2,hour,day,month,weekday,year,minute,second
0,2691,2,1,0,2641,3,5143,7,2,2.0,...,-77.127668,3.8400,71763.0000,17,16,12,2,2015,42,0
1,783,1,1,0,1926,4,1220,9,6,1.0,...,-77.205783,3.9050,7.0733,19,1,5,6,2016,25,0
2,2149,1,1,3,935,0,114,14,12,1.0,...,-77.052688,3.4100,70283.0000,14,16,3,4,2018,30,0
3,291,1,1,10,2727,10,5252,14,2,1.0,...,-77.238475,3.7067,7.3450,17,25,5,4,2018,0,0
4,453,1,1,2,751,3,3005,7,2,1.0,...,-77.154043,3.5143,7.5022,17,23,9,5,2017,40,0


Training the Model

In [34]:
from lightgbm import LGBMClassifier
Classifier=LGBMClassifier()

# fit the model
Classifier.fit(train, y_train)
y_test_pred = Classifier.predict(test)
y_train_pred=Classifier.predict(train)
import sklearn.metrics as metrics
print("Train accuracy LightGBM classifier.:",np.sqrt(metrics.accuracy_score(y_train,y_train_pred)))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train accuracy LightGBM classifier.: 0.9263413898544045


In [35]:
out = {'Fault':y_test_pred}
out= pd.DataFrame(out)
out['Id'] = out.index
out_df={'Id':out['Id'],'Fault':out['Fault']}
out_df=pd.DataFrame(out_df)
print("prediction:\n",out_df)

prediction:
           Id  Fault
0          0      1
1          1      0
2          2      0
3          3      1
4          4      1
...      ...    ...
77230  77230      0
77231  77231      1
77232  77232      0
77233  77233      1
77234  77234      1

[77235 rows x 2 columns]


In [37]:
out_df.to_csv('sample_submission.csv',index = False)